#  Advanced Data Analysis Assignment

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.set(style='darkgrid')
%matplotlib inline

In [36]:
# Load the data

big_mart = pd.read_csv(r"C:\Users\Admin\Rasa_Intern\Files\big_mart_sales.csv")
big_mart.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


In [8]:
# Scenario 1: Complex Filtering with Multiple Conditions


# Compute thresholds
sales_90th = big_mart['Item_Outlet_Sales'].quantile(0.90)
mrp_mean = big_mart['Item_MRP'].mean()
mrp_lower = mrp_mean * 0.90
mrp_upper = mrp_mean * 1.10
visibility_median = big_mart['Item_Visibility'].median()

# Filter data
filtered_items = big_mart[
    (big_mart['Item_Outlet_Sales'] > sales_90th) &
    (big_mart['Item_MRP'].between(mrp_lower, mrp_upper)) &
    (big_mart['Item_Visibility'] < visibility_median)
]

# Display the result
filtered_items.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
54,NCP18,12.15,Low Fat,0.028760,Household,151.4708,OUT017,2007,NaN,Tier 2,Supermarket Type1,4815.0656
844,FDV01,NaN,Regular,0.000000,Canned,154.6314,OUT027,1985,Medium,Tier 3,Supermarket Type3,6515.5188
1173,FDH57,NaN,Low Fat,0.035574,Fruits and Vegetables,131.4284,OUT027,1985,Medium,Tier 3,Supermarket Type3,5404.9644
2355,FDR15,9.30,Regular,0.033432,Meat,153.1314,OUT035,2004,Small,Tier 2,Supermarket Type1,4653.9420
2863,FDS16,NaN,Regular,0.000000,Frozen Foods,145.2760,OUT027,1985,Medium,Tier 3,Supermarket Type3,5273.1360


In [12]:
# Scenario 2: Custom Aggregation Functions

def high_sales_count(series):
    return (series > 5000).sum()

summary_stats = big_mart.groupby('Outlet_Type')['Item_Outlet_Sales'].agg(
    Mean_Sales='mean',
    Std_Dev_Sales='std',
    Q1_Sales=lambda x: x.quantile(0.25),
    Q3_Sales=lambda x: x.quantile(0.75),
    High_Sales_Count=high_sales_count
)

# Display the summary statistics
print(summary_stats)

                    Mean_Sales  Std_Dev_Sales    Q1_Sales    Q3_Sales  \
Outlet_Type                                                             
Grocery Store       339.828500     260.851582   153.79980   458.73620   
Supermarket Type1  2316.181148    1515.965558  1151.16820  3135.91800   
Supermarket Type2  1995.498739    1375.932889   981.55565  2702.64865   
Supermarket Type3  3694.038558    2127.760054  2044.33890  4975.52340   

                   High_Sales_Count  
Outlet_Type                          
Grocery Store                     0  
Supermarket Type1               359  
Supermarket Type2                39  
Supermarket Type3               231  


In [18]:
# Scenario 3: Row-wise Operations and Conditional Tagging

# Calculate thresholds
sales_90 = big_mart['Item_Outlet_Sales'].quantile(0.90)
mrp_75 = big_mart['Item_MRP'].quantile(0.75)

#'Performance_Tag' column
big_mart['Performance_Tag'] = np.where(
    (big_mart['Item_Outlet_Sales'] > sales_90) & (big_mart['Item_MRP'] > mrp_75),
    'High Performer',
    'Average'
)

print(big_mart[['Item_Outlet_Sales', 'Item_MRP', 'Performance_Tag']].head(10))

   Item_Outlet_Sales  Item_MRP Performance_Tag
0          3735.1380  249.8092         Average
1           443.4228   48.2692         Average
2          2097.2700  141.6180         Average
3           732.3800  182.0950         Average
4           994.7052   53.8614         Average
5           556.6088   51.4008         Average
6           343.5528   57.6588         Average
7          4022.7636  107.7622         Average
8          1076.5986   96.9726         Average
9          4710.5350  187.8214  High Performer


In [24]:
# Scenario 4: Pivot Table & Reshaping

# Pivot Table
pivot_table = pd.pivot_table(
    big_mart,
    values='Item_Outlet_Sales',
    index='Item_Type',
    columns='Outlet_Type',
    aggfunc='mean'
)

# Unstack and fill missing values with 0
pivot_filled = pivot_table.fillna(0)

print(pivot_filled)

Outlet_Type            Grocery Store  Supermarket Type1  Supermarket Type2  \
Item_Type                                                                    
Baking Goods              292.082544        2102.021249        1780.368782   
Breads                    381.967442        2346.928355        2022.404489   
Breakfast                 412.831042        2228.334688        1874.670867   
Canned                    352.864879        2348.732970        1867.714110   
Dairy                     341.866589        2392.654389        2083.981362   
Frozen Foods              322.285984        2259.909110        1995.641420   
Fruits and Vegetables     319.750450        2399.947642        1951.642279   
Hard Drinks               243.155708        2231.408037        2247.528955   
Health and Hygiene        365.901818        2127.626505        1876.649134   
Household                 367.286612        2407.403432        2065.970392   
Meat                      387.525864        2260.875454        2

In [38]:
# Scenario 5: Memory Optimization and Dtype Inference

# Memory usage before optimization
memory_before = big_mart.memory_usage(deep=True).sum() / 1024 ** 2  # in MB

# Identifying top 5 memory-consuming columns
top_5_cols = big_mart.memory_usage(deep=True).sort_values(ascending=False).head(5)
print("Top 5 columns by memory usage:\n\n",top_5_cols)

# Optimizing datatypes 
for col in top_5_cols.index:
    if big_mart[col].dtype == 'float64':
        big_mart[col] = big_mart[col].astype('float32')
    elif big_mart[col].dtype == 'int64':
        big_mart[col] = big_mart[col].astype('int32')
    elif big_mart[col].dtype == 'object':
        num_unique = big_mart[col].nunique()
        num_total = len(big_mart[col])
        if num_unique / num_total < 0.5:
            big_mart[col] = big_mart[col].astype('category')

# Memory usage after optimization
memory_after = big_mart.memory_usage(deep=True).sum() / 1024 ** 2  # in MB

print(f"\nMemory usage before optimization: {memory_before:.2f} MB")
print(f"Memory usage after optimization: {memory_after:.2f} MB")
print(f"Reduction: {memory_before - memory_after:.2f} MB")

Top 5 columns by memory usage:

 Outlet_Type             558186
Item_Type               514432
Item_Fat_Content        475240
Outlet_Identifier       468765
Outlet_Location_Type    468765
dtype: int64

Memory usage before optimization: 3.52 MB
Memory usage after optimization: 1.20 MB
Reduction: 2.33 MB


In [40]:
# Scenario 6: MultiIndex Operations

# Set a multi-index
bm_multi = big_mart.set_index(['Outlet_Identifier', 'Item_Type'])

# Retrieve all rows for Outlet_Identifier = 'OUT049'
outlet_rows = bm_multi.loc['OUT049']
print("Rows for Outlet 'OUT049':")
print(outlet_rows.head())

# Retrieve all rows for Item_Type = 'Dairy' across all outlets
# This uses .xs() (cross-section) to slice the second level of the index

dairy_rows = bm_multi.xs('Dairy', level='Item_Type')
print("\nRows for Item_Type 'Dairy':")
print(dairy_rows.head())


Rows for Outlet 'OUT049':
                      Item_Identifier  Item_Weight Item_Fat_Content  \
Item_Type                                                             
Dairy                           FDA15          9.3          Low Fat   
Meat                            FDN15         17.5          Low Fat   
Fruits and Vegetables           FDY07         11.8          Low Fat   
Fruits and Vegetables           FDX32         15.1          Regular   
Breakfast                       FDP49          9.0          Regular   

                       Item_Visibility  Item_MRP  Outlet_Establishment_Year  \
Item_Type                                                                     
Dairy                         0.016047  249.8092                       1999   
Meat                          0.016760  141.6180                       1999   
Fruits and Vegetables         0.000000   45.5402                       1999   
Fruits and Vegetables         0.100014  145.4786                       1999   
Br

In [54]:
# Scenario 7: Exploding Lists and Normalizing Nested Structures

big_mart['promo_tags'] = [
    ['summer', 'festival'], 
    ['winter'], 
    ['summer', 'clearance'], 
    ['festival'], 
    ['clearance', 'winter'], 
] * (len(big_mart) // 5) + [['summer']] * (len(big_mart) % 5)

# Now use .explode() to normalize so each promo tag has its own row
exploded_df = big_mart.explode('promo_tags').reset_index(drop=True)

print(exploded_df.shape)
print("\n\n",exploded_df.isnull().sum(),"\n\n")
exploded_df.head(10)

(13635, 13)


 Item_Identifier                 0
Item_Weight                  2339
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  3852
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales               0
promo_tags                      0
dtype: int64 




,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,promo_tags
0,FDA15,9.300,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380,summer
1,FDA15,9.300,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380,festival
2,DRC01,5.920,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228,winter
3,FDN15,17.500,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700,summer
4,FDN15,17.500,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700,clearance
5,FDX07,19.200,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800,festival
6,NCD19,8.930,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052,clearance
7,NCD19,8.930,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052,winter
8,FDP36,10.395,Regular,0.000000,Baking Goods,51.4008,OUT018,2009,Medium,Tier 3,Supermarket Type2,556.6088,summer
9,FDP36,10.395,Regular,0.000000,Baking Goods,51.4008,OUT018,2009,Medium,Tier 3,Supermarket Type2,556.6088,festival


In [48]:
print("✅ Great job! Submit your completed .ipynb file.")

✅ Great job! Submit your completed .ipynb file.
